In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [5]:
class BlackScholes:
    def __init__(self, time_to_expiry, strike, interest_rate, share_price, vol, dividend):
        self.t = time_to_expiry
        self.k = strike
        self.r = interest_rate
        self.s = share_price
        self.v = vol
        self.q = dividend
        self.d1 = (np.log(share_price / strike) + (interest_rate - dividend + 0.5 * vol**2) * time_to_expiry) / (vol * np.sqrt(time_to_expiry))
        self.d2 = self.d1 - vol * np.sqrt(time_to_expiry) 

    def delta(self, option_type):
        if option_type == 'call':
            return np.exp(-self.q * self.t) * norm.cdf(self.d1)
        
        elif option_type == 'put':
            return np.exp(-self.q * self.t) * (norm.cdf(self.d1) - 1)
    
    def gamma(self):
        return norm.pdf(self.d1) / (self.s * self.v * np.sqrt(self.t)) * np.exp(-self.q * self.t)

    def vega(self):
        return self.s * norm.pdf(self.d1) * np.sqrt(self.t) * np.exp(-self.q * self.t) / 100

    def theta(self, option_type):

        term1 = -(self.s * norm.pdf(self.d1) * self.v) / (2 * np.sqrt(self.t)) * np.exp(-self.q * self.t)
        
        if option_type == 'call':
            term2 = self.q * self.s * np.exp(-self.q * self.t) * norm.cdf(self.d1)
            term3 = self.r * self.k * np.exp(-self.r * self.t) * norm.cdf(self.d2)
            return (term1 - term2 - term3) / 365
        elif option_type == 'put':
            term2 = self.q * self.s * np.exp(-self.q * self.t) * norm.cdf(-self.d1)
            term3 = self.r * self.k * np.exp(-self.r * self.t) * norm.cdf(-self.d2)
            return (term1 + term2 + term3) / 365
    

    def rho(self, option_type):
        if option_type == 'call':
            return self.k * self.t * np.exp(-self.r * self.t) * norm.cdf(self.d2) / 100
        elif option_type == 'put':
            return -self.k * self.t * np.exp(-self.r * self.t) * norm.cdf(-self.d2) / 100
        
    def price(self, option_type):
        if option_type == 'call':
            return (self.s * np.exp(-self.q * self.t) * norm.cdf(self.d1)) - (self.k * np.exp(-self.r * self.t) * norm.cdf(self.d2))
        elif option_type == 'put':
            return (self.k * np.exp(-self.r * self.t) * norm.cdf(-self.d2)) - (self.s * np.exp(-self.q * self.t) * norm.cdf(-self.d1))
    

In [3]:
#Example : call option with strike 70, share price 75, TTE = 1 year, interest rate = 5%, vol = 0.2, dividend = 0.02
S = 70
K = 70
T = 1        
r = 0.05     
vol = 0.2 
q = 0.02     

bs = BlackScholes(T, K, r, S, vol, q)

In [4]:
print("Option Delta: ", bs.delta(option_type='call'))
print("Option Gamma: ", bs.gamma())
print("Option Vega: ", bs.vega())
print("Option Theta: ", bs.theta(option_type='call'))
print("Option Rho: ", bs.rho(option_type='call'))
print("Option Price: ", bs.price(option_type = 'call'))

Option Delta:  0.586851146134764
Option Gamma:  0.027072255364298168
Option Vega:  0.26530810257012205
Option Theta:  -0.014262209449249788
Option Rho:  0.3462067637372565
Option Price:  6.458903855707824
